# ETL para os Dados da Covid-19

## Imports

In [1]:
import pandas as pd
from unidecode import unidecode
import locale
from datetime import date

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

## Carregamento dos Dados

In [2]:
url = 'https://bi.s3.es.gov.br/covid19/MICRODADOS.zip'
# url = '../MICRODADOS.zip'
df = pd.read_csv(url, sep = ';', encoding = 'cp1252')\
    .query('Classificacao == "Confirmados"')\
    .reset_index(drop=True)
df.head()

/home/danilo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2022-01-06,2022-01-03,2022-01-01,2022-01-03,NaN,NaN,NaN,NaN,NaN,Confirmados,...,Sim,Não,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado
1,2022-01-06,2022-01-06,2022-01-06,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
2,2022-01-06,2022-01-06,2022-01-03,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não,Não,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
3,2022-01-06,2022-01-06,2021-12-31,NaN,2022-01-06,NaN,NaN,2022-01-06,NaN,Confirmados,...,Não,Não,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno
4,2022-01-06,2022-01-06,2022-01-05,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno


In [3]:
def print_columns(column_name):
    for coluna_data in list(filter(lambda c: column_name in c, df.columns)):
        print(f'- {coluna_data}')

print('Colunas de Datas:')
print_columns('DataColeta')

print()

print('Colunas de Resultados:')
print_columns('Resultado')

Colunas de Datas:
- DataColeta_RT_PCR
- DataColetaTesteRapido
- DataColetaSorologia
- DataColetaSorologiaIGG

Colunas de Resultados:
- ResultadoRT_PCR
- ResultadoTesteRapido
- ResultadoSorologia
- ResultadoSorologia_IGG


## Visão Geral dos Dados

In [4]:
inicio = df['DataNotificacao'].min()
fim = df['DataNotificacao'].max()

valores_evolucao = ', '.join(df['Classificacao'].unique())

total_casos = df.query('Classificacao == "Confirmados"').shape[0]

total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_obitos_percentual = total_obitos / total_casos

total_recuperados = df.query('Evolucao == "Cura"').shape[0]
total_recuperados_percentual = total_recuperados / total_casos

print(f'''
Visão Geral
===========
- Intervalo de Data dos Dados: {inicio} a {fim}
- Valores Possíveis para a Coluna Evolução: {valores_evolucao}

Totais
======
- Casos Confirmados: {total_casos:n}
- Óbitos: {total_obitos:n} ({total_obitos_percentual:.2%})
- Recuperados: {total_recuperados:n} ({total_recuperados_percentual:.2%})
''')


Visão Geral
- Intervalo de Data dos Dados: 2020-02-29 a 2022-01-06
- Valores Possíveis para a Coluna Evolução: Confirmados

Totais
- Casos Confirmados: 633.080
- Óbitos: 13.346 (2.11%)
- Recuperados: 609.935 (96.34%)



## Definição de Data de Confirmação

In [5]:
# Obtém a data de confirmação do caso, seguindo o mesmo critério do Painel da Covid-19 (data da coleta do exame ou do encerramento do caso)
df['DataConfirmado'] = df['DataColeta_RT_PCR'].combine_first(df['DataColetaTesteRapido'])\
    .combine_first(df['DataColetaSorologia'])\
    .combine_first(df['DataColetaSorologiaIGG'])\
    .combine_first(df['DataEncerramento'])

# Deleta os dados que não possuem data de confirmação.
df = df[df['DataConfirmado'].notna()]
df

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido,DataConfirmado
0,2022-01-06,2022-01-03,2022-01-01,2022-01-03,NaN,NaN,NaN,NaN,NaN,Confirmados,...,Não,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2022-01-03
1,2022-01-06,2022-01-06,2022-01-06,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2022-01-06
2,2022-01-06,2022-01-06,2022-01-03,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2022-01-06
3,2022-01-06,2022-01-06,2021-12-31,NaN,2022-01-06,NaN,NaN,2022-01-06,NaN,Confirmados,...,Não,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2022-01-06
4,2022-01-06,2022-01-06,2022-01-05,NaN,2022-01-06,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2022-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633075,2020-03-13,2020-04-14,2020-03-13,2020-03-15,NaN,NaN,NaN,2020-05-08,NaN,Confirmados,...,Sim,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2020-03-15
633076,2020-03-13,2020-04-14,2020-03-10,2020-03-12,NaN,NaN,NaN,2020-05-07,NaN,Confirmados,...,Sim,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2020-03-12
633077,2020-03-06,2020-04-14,2020-03-04,2020-03-06,NaN,NaN,NaN,2020-05-08,NaN,Confirmados,...,Sim,Não,Não Informado,Não Informado,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2020-03-06
633078,2020-03-06,2020-04-16,2020-03-04,2020-10-08,NaN,NaN,NaN,2020-12-13,NaN,Confirmados,...,Sim,Não,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,2020-10-08


In [6]:
# Deleta os dados que possuem datas antes da pandemia e depois de hoje.
df = df.query(f'DataConfirmado >= "2020-01-01" & DataConfirmado <= "{date.today().isoformat()}"')

confirmado_min = df['DataConfirmado'].min()
confirmado_max = df['DataConfirmado'].max()

print(f'As datas de casos confirmados são de {confirmado_min} a {confirmado_max}')

As datas de casos confirmados são de 2020-01-01 a 2022-01-06


In [7]:
df['DataNotificacao'] = pd.to_datetime(df['DataConfirmado'])
df['DataObito'] = pd.to_datetime(df['DataObito'])
df.sort_values('DataNotificacao', inplace=True)

/tmp/ipykernel_171672/394818174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DataNotificacao'] = pd.to_datetime(df['DataConfirmado'])
/tmp/ipykernel_171672/394818174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DataObito'] = pd.to_datetime(df['DataObito'])
/tmp/ipykernel_171672/394818174.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

## Padroniza nome de municípios e bairros

In [8]:
converte_nome = lambda tipo_regiao: df[tipo_regiao].apply(lambda x: unidecode(str(x)).upper())

df['Municipio'] = converte_nome('Municipio')
df['Bairro'] = converte_nome('Bairro')

/tmp/ipykernel_171672/1535424544.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Municipio'] = converte_nome('Municipio')
/tmp/ipykernel_171672/1535424544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bairro'] = converte_nome('Bairro')


## Cálculo de Confirmados, Óbitos e Curados

### Cria uma série de dados para Datas e uma série para municípios e bairros para fazer um produto cartesiano

In [30]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = pd.concat([df['DataNotificacao'], df[df['DataObito'].notna()]['DataObito']])\
    .drop_duplicates()\
    .reset_index()\
    .rename({0: 'DataNotificacao'}, axis=1)\
    .drop('index', axis=1)

municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)
    
datas['key'] = 0
municipios_bairros['key'] = 0

grupo_base

['DataNotificacao', 'Municipio', 'Bairro']

### Calcula a quantidade de casos cofirmados, óbitos e recuperados por dia para cada bairro

In [31]:
df['Confirmados'] = 1
df['Curas'] = df['Evolucao'].apply(lambda e: 1 if e == 'Cura' else 0)
df['Obitos'] = df['Evolucao'].apply(lambda e: 1 if e == 'Óbito pelo COVID-19' else 0)

df_confirmados_curas = df[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .sum()\
    .reset_index()

df_obitos = df[['Municipio', 'Bairro', 'DataObito', 'Obitos']]\
    .groupby(['Municipio', 'Bairro', 'DataObito'])\
    .sum()\
    .reset_index()\
    .rename({'DataObito': 'DataNotificacao'}, axis=1)

df_counts = df_confirmados_curas.merge(
    df_obitos,
    on=grupo_base,
    how='outer'
).fillna(0)

/tmp/ipykernel_171672/3631408169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Confirmados'] = 1
/tmp/ipykernel_171672/3631408169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Curas'] = df['Evolucao'].apply(lambda e: 1 if e == 'Cura' else 0)
/tmp/ipykernel_171672/3631408169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

### Para os dias que determinado município não possui casos, óbitos ou recuperados, adiciona o valor zero

In [32]:
df_datas_municipios = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_datas_municipios.merge(
    df_counts,
    on=grupo_base,
    how='left'
)

df_counts.fillna(
    {
        'Confirmados': 0,
        'Obitos': 0,
        'Curas': 0,
    },
    inplace=True
)

In [33]:
str_columns_df_counts = ', '.join(df_counts.columns)

menor_data_df_counts = df_counts['DataNotificacao'].min().date().strftime('%d/%m/%Y')
maior_data_df_counts = df_counts['DataNotificacao'].max().date().strftime('%d/%m/%Y')


print(f'''
Análises em df_counts
---------------------
Colunas presentes no DataFrame: {str_columns_df_counts}
Menor data: {menor_data_df_counts}
Maior data: {maior_data_df_counts}
''')


Análises em df_counts
---------------------
Colunas presentes no DataFrame: DataNotificacao, Municipio, Bairro, Confirmados, Curas, Obitos
Menor data: 01/01/2020
Maior data: 06/01/2022



### Calcula a Quantidade Semanal de Casos Confirmados, Óbitos e Recuperados por Bairros

In [34]:
columns_sum = ['Confirmados', 'Obitos', 'Curas']
df_counts_by_week = df_counts.groupby(['Municipio', 'Bairro', pd.Grouper(key='DataNotificacao', freq='W', label='left', closed='left')])[columns_sum]\
    .sum()\
    .reset_index()\
    .sort_values('DataNotificacao')

df_counts_by_week['ConfirmadosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['ObitosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['CurasAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS"').tail(6)

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
494272,SERRA,COLINA DE LARANJEIRAS,2021-11-28,6.0,0.0,6.0,3277.0,37.0,3239.0
494273,SERRA,COLINA DE LARANJEIRAS,2021-12-05,8.0,0.0,8.0,3285.0,37.0,3247.0
494274,SERRA,COLINA DE LARANJEIRAS,2021-12-12,20.0,0.0,20.0,3305.0,37.0,3267.0
494275,SERRA,COLINA DE LARANJEIRAS,2021-12-19,12.0,0.0,12.0,3317.0,37.0,3279.0
494276,SERRA,COLINA DE LARANJEIRAS,2021-12-26,11.0,0.0,2.0,3328.0,37.0,3281.0
494277,SERRA,COLINA DE LARANJEIRAS,2022-01-02,24.0,0.0,1.0,3352.0,37.0,3282.0


In [36]:
df_counts.query('Municipio == "SERRA" & DataNotificacao == "2021-12-30"')['Confirmados'].sum()

53.0

In [ ]:
# df_counts_by_week.to_csv('microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')